In [3]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot
%matplotlib inline

In [4]:
#Hallo Herr Zimmermann,

 

#vielen Dank für Ihre Nachricht.

#Ich habe mir das Notebook angeschaut und eine Lösung vorgeschlagen. Allerdings verstehe ich das Ziel der Methode nicht ganz, von daher müssten Sie einmal prüfen, ob ich es richtig interpretiert habe.

#Mein Vorschlag kann folgendes: für ein Fenster von z.B. 10 Nachrichten vorher oder nachher (Achtung: nachher ist für eine Prediction nicht verwendbar! Zukunftsinformation nennt man das) wird gezählt, wie oft die severity-Klassen 1, 2 und 3 vorkommen. Dies lege ich in drei Spalten ab: pre_no_srv_1, pre_no_srv_2, pre_no_srv_3

#Die vielen Spalten, die durch das „Shiften“ zustande kommen, können Sie auch wieder löschen. Diese sind nur für den Zwischenschritt nötig.

#So bewege ich mich dauerhaft in der Pandas-Welt und von der Performance (Laufzeit und Speicher) gibt es keine Problem.

 

#Ist es das, was Sie erreichen wollten?

 

#Hinsichtlich der Mappings habe ich Ihnen auch noch einen Vorschlag gemacht.

 

#Viele Grüße und ein schönes Wochenende

#Peer Küppers 

In [5]:
#Daten einlesen
df_orig = pd.read_parquet("D:/Eclipse Python/DataScience/res/event_ano.parquet.gzip")

In [6]:
df = df_orig.set_index("event_timestamp")
del df['Unnamed: 0']
df.sort_index(inplace=True)

In [7]:
line_mapping = {}
for k, v in [(l, int(l.split("_")[1])) for l in list(df.an_line.unique())]:
    line_mapping[k] = v

df["an_line_no"] = df.an_line.map(line_mapping)  

In [8]:
cell_mapping = {}
for k, v in [(l, int(l.split("_")[1])) for l in list(df.an_cell.unique())]:
    cell_mapping[k] = v

df["an_cell_no"] = df.an_cell.map(cell_mapping)  

In [9]:
robot_mapping = {}
for k, v in [(l, int(l.split("_")[1])) for l in list(df.an_robot.unique())]:
    robot_mapping[k] = v

df["an_robot_no"] = df.an_robot.map(robot_mapping)  

In [10]:
message_severity_mapping = {"Error":1,"Warning":2,"Information":3}

df["message_severity"] = df.message_severity.apply(lambda x: message_severity_mapping[x] if x in message_severity_mapping.keys() else x)

In [11]:
def dfExtractMethod(df,extract_value):
    df_extract = df.loc[df['an_line_no'] == extract_value]
    return df_extract

In [50]:
def dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternAllMethod(df,no_of_messages):
    dataFrame = pd.DataFrame()
    
    for j in range(0,28):
        df_extract = dfExtractMethod(df,j)
        df_extract = df_extract.reset_index()
                
        for i in range(1,no_of_messages + 1):
            df_extract['msg_svr_pre_{}'.format(i)] = df_extract.shift(i).message_severity

        pre_column_names = ['msg_svr_pre_{}'.format(i) for i in range(1, no_of_messages + 1)]

        for i in range(1, 4):
            df_extract['pre_no_srv_{}'.format(i)] = (df_extract[pre_column_names]==i).sum(axis=1)
        
        dataFrame = dataFrame.append(df_extract)
    return dataFrame

In [51]:
df_ErrorsPerLinesPattern = dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternAllMethod(df,10)
df_ErrorsPerLinesPattern.to_parquet("D:/Eclipse Python/DataScience/res/event_ano_pattern_all.parquet.gzip", compression='gzip')

In [27]:
def dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternSinglesMethod(df,no_of_messages):
    for j in range(0,28):
        df_extract = dfExtractMethod(df,j)
        df_extract = df_extract.reset_index()
        
        for i in range(1,no_of_messages + 1):
            df_extract['msg_svr_pre_{}'.format(i)] = df_extract.shift(i).message_severity

        pre_column_names = ['msg_svr_pre_{}'.format(i) for i in range(1, no_of_messages + 1)]

        for i in range(1, 4):
            df_extract['pre_no_srv_{}'.format(i)] = (df_extract[pre_column_names]==i).sum(axis=1)
    
        df_extract.to_parquet("D:/Eclipse Python/DataScience/res/event_ano_pattern_single_{}.parquet.gzip".format(j),compression='gzip')

In [28]:
dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternSinglesMethod(df,10)

In [25]:
def dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternDataSetMethod(df,no_of_messages):
    for j in range(0,28):
        df_extract = dfExtractMethod(df,j)
        df_extract = df_extract.reset_index()
        
        for i in range(1,no_of_messages + 1):
            print(df_extract.shift(i))

        
        #df_extract.to_parquet("D:/Eclipse Python/DataScience/res/event_ano_pattern_DataSet.parquet.gzip".format(j), compression='gzip')

In [26]:
dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternDataSetMethod(df,10)

          event_timestamp  message_number message_category  message_severity  \
0                     NaN             NaN              NaN               NaN   
1     2016-06-30 12:29:18         10053.0      Operational               3.0   
2     2016-06-30 12:29:18         10156.0      Operational               3.0   
3     2016-06-30 12:29:18         10052.0      Operational               3.0   
4     2016-06-30 12:37:32         10052.0      Operational               3.0   
5     2016-06-30 12:37:33         10156.0      Operational               3.0   
6     2016-06-30 12:37:33         10053.0      Operational               3.0   
7     2016-06-30 12:37:36         50024.0           Motion               2.0   
8     2016-06-30 12:40:14         71201.0  IOCommunication               1.0   
9     2016-06-30 12:40:14         10045.0      Operational               3.0   
10    2016-06-30 12:40:14         71201.0  IOCommunication               1.0   
11    2016-06-30 12:40:14         10045.

            event_timestamp  message_number message_category  \
0                       NaN             NaN              NaN   
1       2016-02-17 05:47:31         10010.0      Operational   
2       2016-02-17 05:47:34         10011.0      Operational   
3       2016-02-17 05:47:45         10010.0      Operational   
4       2016-02-17 05:47:46         10052.0      Operational   
5       2016-02-17 05:47:47         10053.0      Operational   
6       2016-02-17 05:47:47         10156.0      Operational   
7       2016-02-17 05:47:48         10011.0      Operational   
8       2016-02-17 05:47:55         10271.0      Operational   
9       2016-02-17 05:48:00         10052.0      Operational   
10      2016-02-17 05:48:01         10053.0      Operational   
11      2016-02-17 05:48:01         10156.0      Operational   
12      2016-02-17 05:48:08         10271.0      Operational   
13      2016-02-17 05:48:29         10010.0      Operational   
14      2016-02-17 05:48:31         1001

            event_timestamp  message_number message_category  \
0                       NaN             NaN              NaN   
1                       NaN             NaN              NaN   
2                       NaN             NaN              NaN   
3                       NaN             NaN              NaN   
4       2016-02-17 05:47:31         10010.0      Operational   
5       2016-02-17 05:47:34         10011.0      Operational   
6       2016-02-17 05:47:45         10010.0      Operational   
7       2016-02-17 05:47:46         10052.0      Operational   
8       2016-02-17 05:47:47         10053.0      Operational   
9       2016-02-17 05:47:47         10156.0      Operational   
10      2016-02-17 05:47:48         10011.0      Operational   
11      2016-02-17 05:47:55         10271.0      Operational   
12      2016-02-17 05:48:00         10052.0      Operational   
13      2016-02-17 05:48:01         10053.0      Operational   
14      2016-02-17 05:48:01         1015

            event_timestamp  message_number message_category  \
0                       NaN             NaN              NaN   
1                       NaN             NaN              NaN   
2                       NaN             NaN              NaN   
3                       NaN             NaN              NaN   
4                       NaN             NaN              NaN   
5                       NaN             NaN              NaN   
6                       NaN             NaN              NaN   
7       2016-02-17 05:47:31         10010.0      Operational   
8       2016-02-17 05:47:34         10011.0      Operational   
9       2016-02-17 05:47:45         10010.0      Operational   
10      2016-02-17 05:47:46         10052.0      Operational   
11      2016-02-17 05:47:47         10053.0      Operational   
12      2016-02-17 05:47:47         10156.0      Operational   
13      2016-02-17 05:47:48         10011.0      Operational   
14      2016-02-17 05:47:55         1027

            event_timestamp  message_number message_category  \
0                       NaN             NaN              NaN   
1                       NaN             NaN              NaN   
2                       NaN             NaN              NaN   
3                       NaN             NaN              NaN   
4                       NaN             NaN              NaN   
5                       NaN             NaN              NaN   
6                       NaN             NaN              NaN   
7                       NaN             NaN              NaN   
8                       NaN             NaN              NaN   
9                       NaN             NaN              NaN   
10      2016-02-17 05:47:31         10010.0      Operational   
11      2016-02-17 05:47:34         10011.0      Operational   
12      2016-02-17 05:47:45         10010.0      Operational   
13      2016-02-17 05:47:46         10052.0      Operational   
14      2016-02-17 05:47:47         1005

...

            event_timestamp  message_number message_category  \
0                       NaN             NaN              NaN   
1                       NaN             NaN              NaN   
2                       NaN             NaN              NaN   
3                       NaN             NaN              NaN   
4                       NaN             NaN              NaN   
5                       NaN             NaN              NaN   
6       2015-08-17 03:00:20         10230.0      Operational   
7       2015-08-17 03:00:20         10231.0      Operational   
8       2015-08-17 03:00:21         10232.0      Operational   
9       2015-08-17 03:00:49         10231.0      Operational   
10      2015-08-17 03:00:49         10230.0      Operational   
11      2015-08-17 03:00:51         10232.0      Operational   
12      2015-08-17 03:01:30         10230.0      Operational   
13      2015-08-17 03:01:30         10231.0      Operational   
14      2015-08-17 03:01:30         1023

          event_timestamp  message_number message_category  message_severity  \
0                     NaN             NaN              NaN               NaN   
1     2016-03-03 08:38:01         71058.0  IOCommunication               1.0   
2     2016-03-03 08:38:07         71518.0  IOCommunication               2.0   
3     2016-03-03 08:38:07         71276.0  IOCommunication               3.0   
4     2016-03-03 08:38:08         71444.0  IOCommunication               3.0   
5     2016-03-03 08:38:18         71521.0  IOCommunication               1.0   
6     2016-03-03 08:38:18         71520.0  IOCommunication               1.0   
7     2016-03-03 08:38:18         71058.0  IOCommunication               1.0   
8     2016-03-03 08:38:24         71276.0  IOCommunication               3.0   
9     2016-03-03 08:38:24         71518.0  IOCommunication               2.0   
10    2016-03-03 08:38:26         71444.0  IOCommunication               3.0   
11    2016-03-03 08:40:23         10010.

           event_timestamp  message_number message_category  message_severity  \
0                      NaN             NaN              NaN               NaN   
1      2016-03-10 03:59:45         10231.0      Operational               3.0   
2      2016-03-10 03:59:45         10230.0      Operational               3.0   
3      2016-03-10 03:59:46         10232.0      Operational               3.0   
4      2016-03-10 06:30:03         10052.0      Operational               3.0   
5      2016-03-10 06:30:03         10053.0      Operational               3.0   
6      2016-03-10 06:30:03         10156.0      Operational               3.0   
7      2016-03-10 06:30:06         10271.0      Operational               3.0   
8      2016-03-10 06:31:09         10270.0      Operational               3.0   
9      2016-03-10 06:31:58         20481.0           System               2.0   
10     2016-03-10 06:31:58         10125.0      Operational               3.0   
11     2016-03-10 06:31:58  

           event_timestamp  message_number message_category  message_severity  \
0                      NaN             NaN              NaN               NaN   
1                      NaN             NaN              NaN               NaN   
2                      NaN             NaN              NaN               NaN   
3                      NaN             NaN              NaN               NaN   
4                      NaN             NaN              NaN               NaN   
5                      NaN             NaN              NaN               NaN   
6                      NaN             NaN              NaN               NaN   
7                      NaN             NaN              NaN               NaN   
8                      NaN             NaN              NaN               NaN   
9      2016-03-10 03:59:45         10231.0      Operational               3.0   
10     2016-03-10 03:59:45         10230.0      Operational               3.0   
11     2016-03-10 03:59:46  

           event_timestamp  message_number message_category  message_severity  \
0                      NaN             NaN              NaN               NaN   
1                      NaN             NaN              NaN               NaN   
2                      NaN             NaN              NaN               NaN   
3                      NaN             NaN              NaN               NaN   
4                      NaN             NaN              NaN               NaN   
5                      NaN             NaN              NaN               NaN   
6      2016-03-16 04:04:19         10232.0      Operational               3.0   
7      2016-03-16 04:04:19         10230.0      Operational               3.0   
8      2016-03-16 04:04:19         10231.0      Operational               3.0   
9      2016-03-16 05:05:49         10010.0      Operational               3.0   
10     2016-03-16 05:05:50         10011.0      Operational               3.0   
11     2016-03-16 05:06:55  